# Предобработка

In [ ]:
!git lfs install
!git clone https: // huggingface.co/facebook/nllb-200-distilled-600M

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("SirNeural/flan_v2", cache_dir="flan_v2", data_files="cot_*_train.jsonl.gz")

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True)
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
None

In [ ]:
article = "Hi cutie"
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"], max_length=30)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

In [4]:
train_dataset = train_dataset['train']

In [5]:
train_dataset

Dataset({
    features: ['inputs', 'targets', 'task'],
    num_rows: 373681
})

In [7]:
from tqdm.auto import tqdm

tasks = []
for i in tqdm(range(len(train_dataset))):
    t = train_dataset[i]['inputs'].find(':')
    if train_dataset[i]['inputs'][:t] not in tasks:
        tasks.append(train_dataset[i]['inputs'][:t])
tasks[:20]

  0%|          | 0/373681 [00:00<?, ?it/s]

['Q',
 'QUESTION',
 'Question',
 'Student A',
 '[Q] Premise',
 '"Kane earned a degree in English literature at Truman State University, where he played baseball." Is the above claim true? Options',
 'Next Question',
 'Student asked',
 '[QUESTION] Premise',
 'Is the following statement true? "Prime Minister of Pakistan sings songs in his car at midnight each Tuesday." no I think the sentence is false because it is highly specific and there is no reason to think it is true without any evidence. There is no evidence of him doing this. Is the following a factual statement? "Some animals apparently cannot die of Old age." Options',
 'Jax',
 '[QUESTION] Given the sentence "Ballerina in black dress posed in an abandon building." is it true that "A ballerina in a black dress poses in a building."? Ballerina in an abandon building is a ballerina in a building. The answer is yes. [QUESTION] Given the sentence "Many people are rollerblading down the street." is it true that "The group of friends 

In [8]:
sentences = []

for i in tqdm(range(len(train_dataset))):
    sentences.append(train_dataset[i]['inputs'])
    sentences.append(train_dataset[i]['targets'])

  0%|          | 0/373681 [00:00<?, ?it/s]

### Перевод

In [11]:
import jsonlines
from torch.cuda.amp import autocast


def predict(
        model_name,
        data_,
        max_source_tokens_count=520,
        max_target_tokens_count=520,
        use_cuda=True,
        batch_size=128
):
    russian_samples = []
    inputs = data_

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True, src_lang="eng_Latn")
    device = torch.device("cuda:1") if use_cuda else torch.device("cpu")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=True).to(device)
    model.eval()
    predictions = []
    with autocast(dtype=torch.float16):
        with torch.no_grad():
            for i in tqdm(range(0, len(sentences), batch_size)):
                batch = sentences[i:i + batch_size]
                input_ids = tokenizer.prepare_seq2seq_batch(
                    batch,
                    return_tensors="pt",
                    padding="max_length",
                    truncation=True,
                    max_length=max_source_tokens_count
                )["input_ids"].to(device)

                output_ids = model.generate(
                    input_ids=input_ids,
                    max_length=max_target_tokens_count,
                    forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"]
                )
                decoded_output = tokenizer.batch_decode(output_ids, skip_special_tokens=True,
                                                        clean_up_tokenization_spaces=False)
                russian_samples.extend(decoded_output)
                if (i // batch_size) % 10 == 0:
                    with jsonlines.open('flan_traslation_v2.jsonl', mode='w') as writer:
                        writer.write(russian_samples)
    return russian_samples

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

russian_samples = predict("facebook/nllb-200-distilled-600M", sentences)

In [ ]:
with jsonlines.open('flan_traslation_v2.jsonl', mode='w') as writer:
    writer.write(russain_samples)

In [5]:
import jsonlines

with jsonlines.open('flan_traslation_v2.jsonl') as reader:
    f = reader.read()
f[:10]

['Вопрос: Это список самых продаваемых видеоигр всех времен. Самой продаваемой видеоигрой на сегодняшний день является Tetris, видеоигровая игра с тиловой пазлой, которая первоначально вышла для Electronika 60 в феврале 1984 года, а затем стала популярной после выхода на Game Boy в 1989 году. Игра была перенесена на широкий спектр платформ и продана более чем на 170 миллионов единиц, включая 100 миллионов платных загрузок на мобильных телефонах и 35 миллионов продаж для версии Game Boy. Minecraft является единственной другой видеоигрой, которая продала более 100 миллионов единиц, с 122 миллионами единиц, продаваемыми по состоянию на февраль 2017 года. Ответ, если не считать мобильных продаж, является самым продаваемым видеоигрой всех времен, продается на Wii Sports с 82,81 миллионами единиц, в то время как Tetris является четвертым с продажей 70 миллионов единиц.',
 'Джеки Уорд (род. Джаклин Макдональд, 1941), более известный как Робин Уорд. Как объяснить ответ?',
 'Вопрос: Салли стоял

In [6]:
from tqdm.auto import tqdm

ds = []
t = {}
for i in tqdm(range(0, len(f), 2)):
    t = {'inputs': f[i], 'target': f[i + 1]}
    ds.append(t)
ds[:10]

100%|██████████| 373184/373184 [00:00<00:00, 985082.69it/s] 


[{'inputs': 'Вопрос: Это список самых продаваемых видеоигр всех времен. Самой продаваемой видеоигрой на сегодняшний день является Tetris, видеоигровая игра с тиловой пазлой, которая первоначально вышла для Electronika 60 в феврале 1984 года, а затем стала популярной после выхода на Game Boy в 1989 году. Игра была перенесена на широкий спектр платформ и продана более чем на 170 миллионов единиц, включая 100 миллионов платных загрузок на мобильных телефонах и 35 миллионов продаж для версии Game Boy. Minecraft является единственной другой видеоигрой, которая продала более 100 миллионов единиц, с 122 миллионами единиц, продаваемыми по состоянию на февраль 2017 года. Ответ, если не считать мобильных продаж, является самым продаваемым видеоигрой всех времен, продается на Wii Sports с 82,81 миллионами единиц, в то время как Tetris является четвертым с продажей 70 миллионов единиц.',
  'target': 'Джеки Уорд (род. Джаклин Макдональд, 1941), более известный как Робин Уорд. Как объяснить ответ?'}

In [11]:
with jsonlines.open('flan_traslation_v22.jsonl', mode='w') as writer:
    writer.write(ds)

In [8]:
import json

with open('flan_traslation_v22.jsonl', 'w') as f:
    for item in ds:
        json.dump(item, f)
        f.write('\n')

In [12]:
def jsonl_reader(file_name):
    with open(file_name, "r") as file:
        reader = jsonlines.Reader(file)
        for line in reader.iter():
            print(line)